In [28]:
import respy as rp
import pandas as pd
import numpy as np

from auxiliary import _get_choices_occupations

In [30]:
params, options, data = rp.get_example_model("kw_94_two")

In [32]:
entries_to_remove = list()
for key_ in options["covariates"].keys():
    if "exp_" in key_:
        entries_to_remove.append(key_)

for key_ in entries_to_remove:
    options["covariates"].pop(key_, None)

_, occupations = _get_choices_occupations(params)

for occupation in occupations:
    options["covariates"][f"exp_{occupation}_square"] = f"exp_{occupation} ** 2"

In [33]:
options

{'estimation_draws': 200,
 'estimation_seed': 500,
 'estimation_tau': 500,
 'interpolation_points': -1,
 'n_periods': 40,
 'simulation_agents': 1000,
 'simulation_seed': 132,
 'solution_draws': 500,
 'solution_seed': 1,
 'monte_carlo_sequence': 'random',
 'core_state_space_filters': ["period > 0 and exp_{choices_w_exp} == period and lagged_choice_1 != '{choices_w_exp}'",
  "period > 0 and exp_a + exp_b + exp_edu == period and lagged_choice_1 == '{choices_wo_exp}'",
  "period > 0 and lagged_choice_1 == 'edu' and exp_edu == 0",
  "lagged_choice_1 == '{choices_w_wage}' and exp_{choices_w_wage} == 0",
  "period == 0 and lagged_choice_1 == '{choices_w_wage}'"],
 'covariates': {'constant': '1',
  'not_edu_last_period': "lagged_choice_1 != 'edu'",
  'exp_a_square': 'exp_a ** 2',
  'exp_b_square': 'exp_b ** 2'}}